In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
# for the sake of reproducibility 

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# RNN recap

<img src="./pics/rnn.png" width="90%">

Simplest RNN consisting of 1 layer receives $x_{(t)}$ and could be written as:

$$y_{(t)} = \phi (x_{(t)}^T \cdot w_x + y_{(t-1)}^T \cdot w_y + b)$$

where 
* $x(t)$ -- input vector at time step _t_ 
* $y(t)$ -- output vector at time step _t_
* $w_x$ -- weights vector for input 
* $w_y$ -- weights vector for output
* $y(t-1)$ -- output vector at previous time step; for 0th step it's zero vector
* $b$ -- bias
* $\phi$ -- some activation function, i.e. ReLU


Also we should mention **hidden_state** ( $h(t)$ ) -- it's recurrent cell memory.

In general case $h_{(t)} = f(h_{(t-1)}, x_{(t)})$, but also $y{(t)} = f(h{(t-1)}, x{(t)})$. So in this case $h(t) == y(t)$, but in practice more complex architectures are used, where **hidden_state** is not equal to the RNN output.

------

## Lets write simple RNN
To write RNN we need to make few improvements to formula.

Lets say that we have not only one vector $x_{(t)}$ as input, but a few vectors in mini-batch $X_{(t)}$ of size $m$ . So all consequent computaions will be in matrix form.

$$ Y_{(t)} = \phi(X_{(t)} \cdot W_x + Y_{(t-1)} \cdot W_y + b) = \phi([X_{(t)} Y_{(t-1)}] \cdot W + b) $$
where
$$ W = [W_x W_y]^T $$

*It's matrix concatination in square brackets

Dimentions:
* $Y_{(t)}$ -- matrix [$m$ x n_neurons]
* $X_{(t)}$ -- matrix [$m$ x n_features]
* $b$ -- vector of size `n_neurons`
* $W_x$ -- input weights of size [n_features x n_neurons]
* $W_y$ -- output weights of size [n_neurons x n_neurons]

In [ ]:
reset_graph()

n_features = 3
n_neurons = 5

# two time steps
X0 = tf.placeholder(tf.float32, [None, n_features])
X1 = tf.placeholder(tf.float32, [None, n_features])

Wx = tf.Variable(tf.random_normal(shape=[n_features, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

# tanh as phi
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

In [ ]:
# mini-batches of size 4
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])  # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])  # t = 1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [ ]:
Y0_val

In [ ]:
Y1_val

## Task

Make the same computation using only one matrix multiplication.

In [ ]:
reset_graph()

X0 = tf.placeholder(tf.float32, [None, n_features])
X1 = tf.placeholder(tf.float32, [None, n_features])

< your code here >

with tf.Session() as sess:
    init.run()
    Y0_val_1, Y1_val_1 = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

# Dynamic_rnn

In TensorFlow there is a function `tf.contrib.rnn.static_rnn` which create for each time step (unrolling) specific cell of desired type. Our implementation follows `tf.contrib.rnn.BasicRNNCell`. This implementation has such a drawback - we could need a lot of memory for long sequences. And because we want to work with such sequences we need to allocate a lot of memory at once. But in TF there is another option -- `dynamic_rnn`, where memory is allocated dynamically for each provided sequence, acording to its lenght.

Lets rewrite the code with `dynamic_rnn`.

In [ ]:
n_steps = 2
n_features = 3
n_neurons = 5

reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_features])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
seq_length = tf.placeholder(tf.int32, [None])
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
                                    sequence_length=seq_length)

In [ ]:
init = tf.global_variables_initializer()
X_batch = np.array([
        # step 0     step 1
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2 (padded with zero vectors)
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

# sequence lengths
seq_length_batch = np.array([2, 1, 2, 2])

In [ ]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run(
        [outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [ ]:
print(outputs_val.shape)
print(states_val.shape)

In [ ]:
# for the second sample there are zeros in output 
print(outputs_val)

In [ ]:
# but in state there are not
print(states_val)

# Name generation

Lets try to do something useful with our RNNs.

_Teaser:_

* It is hard to choose a name for a variable. But its much harder to choose a name for a person.
  So lets make neural net do it instead!
* Dataset consists of 8 thousand people names from different cultures all around the world.
* Our toy task be to train a model to generate a name.

In [ ]:
start_token = " "

with open("names") as f:
    names = f.readlines()
    names = [start_token + name.lower() for name in names]

In [ ]:
print('n samples = ', len(names))
for x in names[::1000]:
    print(x.strip().capitalize())

# Text processing

Lets take all the latters disregarding a case + symbol ')' for the end of a name

In [ ]:
token_set = set()
for name in names:
    for letter in name:
        token_set.add(letter)


token_set.add(')')
tokens = list(token_set)
tokens.sort()

print('n_tokens = ', len(tokens))

In [ ]:
token_to_id = {t: i for i, t in enumerate(tokens)}

id_to_token = {i: t for i, t in enumerate(tokens)}

## Name length distribution

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(list(map(len, names)))

# max length of a name in this dataset
MAX_LEN = min([60, max(list(map(len, names)))])-1

print(MAX_LEN)

### Convert symbols to their ids

In [ ]:
names_ix = list(map(lambda name: list(map(token_to_id.get, name + ')')), names))


for i in range(len(names_ix)):
    names_ix[i] = names_ix[i][:MAX_LEN+1] #crop too long
    
    if len(names_ix[i]) < MAX_LEN+1:
        names_ix[i] += [token_to_id[" "]]*(MAX_LEN+1 - len(names_ix[i])) #pad too short
        
assert len(set(map(len, names_ix))) == 1

names_ix = np.array(names_ix)

In [ ]:
names_ix[:10]

## Batch generator

In [ ]:
def sample_batch(data, batch_size):
    
    rows = data[np.random.randint(0, len(data), size=batch_size)]
    x = rows[:, :-1]
    y = rows[:, 1:]
    
    count = lambda r: np.sum([id_to_token[t] != ' ' for t in r])
    lengths = list(map(count, x))
    
    return x, y, lengths

In [ ]:
x, y, length = sample_batch(names_ix, 10)
y.shape

In [ ]:
x

In [ ]:
y

In [ ]:
length

# Network inputs

In [ ]:
reset_graph()

X = tf.placeholder(tf.int32, [None, None], name= 'X')
y = tf.placeholder(tf.int32, [None, None], name = 'y')
lengths = tf.placeholder(tf.int32, [None], name = 'lengths')
learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])

In [ ]:
n_neurons = 60
embedding_size = 8
vocabulary_size = len(tokens)

n_steps = MAX_LEN # this is number of unrollings


embedding_mtx = < create matrix of embeddings >

embed = < embed the input sequence >


cell = tf.contrib.rnn.BasicRNNCell(< choose params >)
rnn_outputs, states = tf.nn.dynamic_rnn(< choose params >)

pred_logits = < make logits >

labels_one_hot = < make targets >

stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=labels_one_hot,
    logits=pred_logits)
    
loss = tf.reduce_mean(stepwise_cross_entropy)

pred_probas = tf.nn.softmax(pred_logits)

# sampling predictions of size [batch_size, num_steps]
prediction = tf.argmax(pred_probas, axis=2)

train_op = tf.train.AdamOptimizer(learning_rate_ph).minimize(loss)

# get the probability distribution for the last symbol
# that will be needed for generation
last_word_probas = pred_probas[:, -1]

### Which params in network are being trained?

In [ ]:
tf.trainable_variables()

### Function to generation a name

* Lets take seed phrase
* Predicting next token
* Next token is being sampled from model predicted distribution 
* Token is added to seed phrase
* Repeat (from step 2)

In [ ]:
def generate_sample(sess, seed_phrase=None, N=MAX_LEN, n_snippets=1):
    
    if seed_phrase is None:
        seed_phrase = ' '
    elif seed_phrase[0].isalpha():
        seed_phrase = ' ' + seed_phrase
    seed_phrase = seed_phrase.lower()
    seed_phrase = np.array([token_to_id[tok] for tok in seed_phrase])
    L = len(seed_phrase)
    snippets = []
    for _ in range(n_snippets):
        x = np.zeros(N)
        x[:len(seed_phrase)] = seed_phrase
        for n in range(N - L):
            feed_dict = {X: x[:L + n].reshape([1, -1]), lengths: [len(x)]}
            p = sess.run(last_word_probas, feed_dict=feed_dict).reshape(-1)
            ix = np.random.choice(np.arange(len(tokens)), p=p)
            x[L + n] = ix
        snippet = ''.join([id_to_token[idx] for idx in x])
        if ')' in snippet:
            upto = snippet.index(')')
            snippet = snippet[:upto]
        snippets.append(snippet.strip().capitalize())
    return snippets

In [ ]:
def print_pred(y_pred, k = 3):
    for i in range(k):
        print("".join( [id_to_token[t] for t in y_pred[i,:]]))

## Training

In [ ]:
s = tf.Session()
    
s.run(tf.global_variables_initializer())

n_epochs = 5
batches_per_epoch = 500
batch_size = 10
lr = 1e-2
for epoch in range(n_epochs):

    print(">>Generated: ", generate_sample(s, n_snippets=6))
    print("-------\n")
    avg_cost = 0
    for batch in range(batches_per_epoch):
        x_, y_, len_ = sample_batch(names_ix, batch_size)

        _, iloss, y_pred = s.run([train_op, loss, prediction], {X: x_,
                                                                y: y_,
                                                                lengths: len_,
                                                                learning_rate_ph: lr})
        avg_cost += iloss

    print("EPOCH: ", epoch)
    print("AVERAGE LOSS: ", avg_cost/batches_per_epoch)
    print(">>Predicted: ")
    print_pred(y_pred)

print(">>Generated: ", generate_sample(s, n_snippets=6))

In [ ]:
generate_sample(s, seed_phrase='Puti', n_snippets=12)

In [ ]:
generate_sample(s, seed_phrase='Q', n_snippets=6)

In [ ]:
generate_sample(s, seed_phrase='Eug', n_snippets=6)

In [ ]:
generate_sample(s, seed_phrase='Lu', n_snippets=19)

## Bonus part
### Do more interesting stuff

* Multi-layer (MultiRNNCell);
* Try some other cell types: LSTM, GRU;
* Try to generate tweet, using [this](http://study.mokoron.com) dataset.
